# [TEST]-download_specificdate_osmnx_network

This notebook was used to test the download of specific dates (__historic download__) of the OSMnx network.

For this notebook, function aup/data.py/create_osmnx_network() was adapted so that it accepts a specific query date. (specific_date arg)

## __Results:__
Downloading historic data is possible, but __data is not always available__, specially true in small cities.
Data availability can be checked at https://overpass-turbo.eu/ using Overpass queries such as:
* __Query date:__ 01 january 2010 (Must be in form of yyyy-mm-ddThh:mm:ssZ)
* __La Piedad bbox:__ Polygon N:20.62273, S:20.18798, E-101.5949, W-102.18026

#### __OVERPASS-TURBO QUERY EXAMPLE:__
##### (OPEN CELL AND COPY THE FOLLOWING TO LINK WITHOUT ALL THE '--')
--[out:json][timeout:90][date:"2010-01-01T12:00:00Z"];
--(
--  node(20.18798,-102.18026,20.62273,-101.5949);
--  way(20.18798,-102.18026,20.62273,-101.5949);
--  relation(20.18798,-102.18026,20.62273,-101.5949);
--);
--out body;
-->;
out skel qt;
##### (OPEN CELL AND COPY THE FOLLOWING TO LINK WITHOUT ALL THE '--')


## __Conclusion:__
__Downloading historic data from OSMnx possible, but data availability difference between big and small cities increases as time goes back.__

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Load all available cities

In [2]:
# gdf_mun
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

city_list = list(metro_gdf.city.unique())

# Show
print(metro_gdf.shape)
print(len(city_list))
print(city_list)
metro_gdf.head(1)

(367, 6)
71
['Aguascalientes', 'Ensenada', 'Mexicali', 'Tijuana', 'La Paz', 'Los Cabos', 'Campeche', 'Laguna', 'Monclova', 'Piedras Negras', 'Saltillo', 'Colima', 'Tapachula', 'Tuxtla', 'Chihuahua', 'Delicias', 'Juarez', 'CDMX', 'ZMVM', 'Durango', 'Celaya', 'Guanajuato', 'Leon', 'Irapuato', 'Acapulco', 'Chilpancingo', 'Pachuca', 'Tulancingo', 'Guadalajara', 'Vallarta', 'Piedad', 'Toluca', 'Morelia', 'Zamora', 'Uruapan', 'Cuautla', 'Cuernavaca', 'Tepic', 'Monterrey', 'Oaxaca', 'Puebla', 'San Martin', 'Tehuacan', 'Queretaro', 'Cancun', 'Chetumal', 'Playa', 'SLP', 'Culiacan', 'Los Mochis', 'Mazatlan', 'Guaymas', 'Ciudad Obregon', 'Hermosillo', 'Nogales', 'Villahermosa', 'Victoria', 'Matamoros', 'Nuevo Laredo', 'Reynosa', 'Tampico', 'Tlaxcala', 'Coatzacoalcos', 'Cordoba', 'Minatitlan', 'Orizaba', 'Poza Rica', 'Veracruz', 'Xalapa', 'Merida', 'Zacatecas']


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0...",Aguascalientes


## __Current, present__ OSMnx test

In [4]:
# RUN ALL OTHER CITIES
city_list = ['Piedad']
skip_city_list = []
nodes_processed_city_list = []
edges_processed_city_list = []
i = 0
k = len(city_list)

for city in city_list:
    if city not in skip_city_list:
        if (city not in nodes_processed_city_list) and (city not in edges_processed_city_list):
            print("--"*40)
            i = i + 1
            print(f"Starting city {i}/{k}: {city}")
        
            # Load area of interest
            city_gdf = metro_gdf.loc[metro_gdf.city == city]
            aoi = city_gdf.dissolve()
            
            # Create Network
            G,nodes,edges = aup.create_osmnx_network(aoi, how='from_bbox', network_type='all_private')

            nodes['city'] = city
            edges['city'] = city

--------------------------------------------------------------------------------
Starting city 1/1: Piedad
Extracted min and max coordinates from the municipality. Polygon N:20.62273, S:20.18798, E-101.5949, W-102.18026.
Created OSMnx graph from bounding box.
Converted OSMnx graph to 22199 nodes and 56195 edges GeoDataFrame.
Added column width for edges.
Filtered columns.
Column: osmid in nodes gdf, has a list in it, the column data was converted to string.
Column: lanes in nodes gdf, has a list in it, the column data was converted to string.
Column: name in nodes gdf, has a list in it, the column data was converted to string.
Column: highway in nodes gdf, has a list in it, the column data was converted to string.
Column: maxspeed in nodes gdf, has a list in it, the column data was converted to string.


In [5]:
# Show
print(nodes.shape)
nodes.head(2)

(22199, 5)


,x,y,street_count,geometry,city
osmid,,,,,
294112061,-101.769838,20.386650,3,POINT (-101.76984 20.38665),Piedad
1270369479,-101.642189,20.313361,3,POINT (-101.64219 20.31336),Piedad


In [6]:
# Show
print(edges.shape)
edges.head(2)

(56195, 16)


osmid  oneway lanes                  name   
u         v          key                                                  
294112061 6453194108 0    893281259    True     2  Carretera Federal 90  \
          6453194136 0    897512872   False  <NA>                  <NA>   

                               highway maxspeed   length   
u         v          key                                   
294112061 6453194108 0           trunk     <NA>  530.615  \
          6453194136 0    unclassified     <NA>   85.062   

                                                                   geometry   
u         v          key                                                      
294112061 6453194108 0    LINESTRING (-101.76984 20.38665, -101.76933 20...  \
          6453194136 0    LINESTRING (-101.76984 20.38665, -101.76983 20...   

                         bridge     ref junction tunnel access  width service   
u         v          key                                                        
294112061 6453194108 0      NaN  MEX 90      NaN    NaN    NaN    NaN     NaN  \
          6453194136 0      NaN     NaN      NaN    NaN    NaN    NaN     NaN   

                            city  
u         v          key          
294112061 6453194108 0    Piedad  
          6453194136 0    Piedad

## __Specific date__ OSMnx Test

In [7]:
# From OSMnx module: Date must be in form of yyyy-mm-ddThh:mm:ssZ
# Example: '[out:json][timeout:90][date:"2019-10-28T19:20:00Z"]'

# For Aguascalientes:
#'[out:json][timeout:90][date:"2010-01-01T00:00:00Z"]' --> ValueError: some edges missing nodes, possibly due to input data clipping issue
#'[out:json][timeout:90][date:"2015-01-01T12:00:00Z"]' --> Funcionó
#'[out:json][timeout:90][date:"2010-06-01T12:00:00Z"]' --> ValueError: some edges missing nodes, possibly due to input data clipping issue

# After those tests, https://overpass-turbo.eu/ was used to confirm data unavailability.

In [11]:
city_list = ['Aguascalientes']
skip_city_list = []
nodes_processed_city_list = []
edges_processed_city_list = []
i = 0
k = len(city_list)

for city in city_list:
    if city not in skip_city_list:
        if (city not in nodes_processed_city_list) and (city not in edges_processed_city_list):
            print("--"*40)
            i = i + 1
            print(f"Starting city {i}/{k}: {city}")
        
            # Load area of interest
            city_gdf = metro_gdf.loc[metro_gdf.city == city]
            aoi = city_gdf.dissolve()
            
            # Create Network
            G,nodes,edges = aup.create_osmnx_network(aoi, 
                                                     how='from_bbox', 
                                                     network_type='all_private', 
                                                     specific_date=(True,'[out:json][timeout:90][date:"2010-01-01T12:00:00Z"]'))

            nodes['city'] = city
            edges['city'] = city

--------------------------------------------------------------------------------
Starting city 1/1: Aguascalientes
Extracted min and max coordinates from the municipality. Polygon N:22.10033, S:21.62227, E-102.06451, W-102.59887.


ValueError: some edges missing nodes, possibly due to input data clipping issue

In [9]:
# Show
print(nodes.shape)
nodes.head(2)

(22199, 5)


,x,y,street_count,geometry,city
osmid,,,,,
294112061,-101.769838,20.386650,3,POINT (-101.76984 20.38665),Piedad
1270369479,-101.642189,20.313361,3,POINT (-101.64219 20.31336),Piedad


In [10]:
# Show
print(edges.shape)
edges.head(2)

(56195, 16)


osmid  oneway lanes                  name   
u         v          key                                                  
294112061 6453194108 0    893281259    True     2  Carretera Federal 90  \
          6453194136 0    897512872   False  <NA>                  <NA>   

                               highway maxspeed   length   
u         v          key                                   
294112061 6453194108 0           trunk     <NA>  530.615  \
          6453194136 0    unclassified     <NA>   85.062   

                                                                   geometry   
u         v          key                                                      
294112061 6453194108 0    LINESTRING (-101.76984 20.38665, -101.76933 20...  \
          6453194136 0    LINESTRING (-101.76984 20.38665, -101.76983 20...   

                         bridge     ref junction tunnel access  width service   
u         v          key                                                        
294112061 6453194108 0      NaN  MEX 90      NaN    NaN    NaN    NaN     NaN  \
          6453194136 0      NaN     NaN      NaN    NaN    NaN    NaN     NaN   

                            city  
u         v          key          
294112061 6453194108 0    Piedad  
          6453194136 0    Piedad